In [59]:
import pandas as pd
import nltk
import string
import itertools
import numpy as np

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter

In [60]:
pizza=pd.read_csv('pizza_raw.csv',index_col=0)

In [61]:
pizza.head()

,review_id,stars_x,text,name,city,state,stars_y,attributes,categories,hours
0,hfXVxrlD6fr1kXfz6sR13Q,2,Parking is tough to get here so I don't get to...,Original Hot Dog Shop,Pittsburgh,PA,3.5,"{'Caters': 'False', 'RestaurantsTakeOut': 'Tru...","Hot Dogs, Burgers, Restaurants, Pizza","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'..."
1,EDQpLLFiAxeSxWI5ipwzBQ,4,The server had an extensive knowledge of wines...,Humble Wine Bar,Lakewood,OH,4.0,"{'HasTV': 'True', 'Alcohol': ""u'full_bar'"", 'B...","Food, Tapas/Small Plates, Vegan, Desserts, Res...","{'Monday': '16:0-22:0', 'Tuesday': '16:0-22:0'..."
2,RWmT90rP7wYo57dG1cyvww,5,Fast Forward\n\nEvery trip from NYC to Pitt in...,Aiello's Pizza,Pittsburgh,PA,4.0,"{'BikeParking': 'True', 'Alcohol': ""u'beer_and...","Pizza, Italian, Restaurants, Beer Bar, Bars, N...","{'Monday': '11:0-1:0', 'Tuesday': '11:0-1:0', ..."
3,0jduztItUhOh3FzKGjMxZQ,5,By far my favorite Cleveland/Akron restaurant!...,Russo's,Peninsula,OH,4.0,"{'RestaurantsReservations': 'True', 'Restauran...","Italian, Sandwiches, Pizza, Bars, Cocktail Bar...","{'Tuesday': '17:0-21:0', 'Wednesday': '17:0-21..."
4,sxfmzhMvZ8LlI1x4fhNjKg,4,All food is good here I usually get the Italia...,Tnt Dynamite Pizza & Hoagies,Springdale,PA,3.0,"{'RestaurantsTakeOut': 'True', 'HasTV': 'True'...","Restaurants, Pizza",NaN


In [62]:
text = list(pizza.text)
# Transform all words in text into lower cases
text = [a.lower() for a in text]

# Remove all punctuations in text
text = ["".join([char for char in a if char not in string.punctuation]) for a in text]

In [63]:
# Extract words in each review text
words = [nltk.word_tokenize(a) for a in text]

# Filter common Enlish words
stop_words = stopwords.words('english')
stop_words = ["".join([char for char in a if char not in string.punctuation]) for a in stop_words]
filtered_words = [[word for word in a if word not in stop_words] for a in words]

# Lemmatize words
lmtzr = WordNetLemmatizer()
lemmatized_words = [[lmtzr.lemmatize(x) for x in a] for a in filtered_words]

In [64]:
# Count Words Frequency
word_list = list(itertools.chain.from_iterable(lemmatized_words))
freq = pd.DataFrame.from_dict(Counter(word_list), orient='index')
freq.sort_values(by=0, ascending=False)

,0
pizza,113385
good,45751
place,42345
food,39252
great,33372
...,...
snackindividual,1
casualstill,1
herbsparmesan,1
veganhalf,1


In [65]:
# Create a word dictionary
dictionary = list(freq[freq[0] > 500].index) ## Select words that occur more than 500 times.
len(dictionary)

1135

In [66]:
# Word embedding
word_embedding = np.zeros([len(lemmatized_words), len(dictionary)])
for i,x in enumerate(lemmatized_words):
    for j,y in enumerate(dictionary):
        word_embedding[i,j] = sum([z == y for z in x])

In [67]:
word_embedding

array([[1., 1., 5., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [68]:
df = pd.concat([pizza, pd.DataFrame(word_embedding, columns=dictionary)], axis=1)
df = df.drop(columns=["review_id","text"])
df.head()

In [ ]:
df.to_csv('WordEmbeddingPy.csv')